In [223]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [224]:
df = pd.read_csv("C:\\Users\\Ergney\\qwe\\data.csv")

In [225]:
df = df.rename(columns={'conversion':"target", "offer":"treatment"})

In [226]:
df["treatment"].loc[df["treatment"] == 'No Offer'] = 0

C:\Users\Ergney\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [227]:
df["treatment"].loc[df["treatment"] != 0] = 1

In [228]:
df["treatment"] = pd.to_numeric(df["treatment"], errors='coerce')

In [229]:
df["treatment"].value_counts()

1    42694
0    21306
Name: treatment, dtype: int64

In [230]:
features = df.columns[0:-1]
x_train, x_test, y_train, y_test = train_test_split(df[features], df["target"], test_size=0.3, random_state=42)

In [231]:
x_train

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment
9656,7,434.35,1,0,Urban,1,Web,0
63037,1,376.59,1,0,Surburban,0,Multichannel,1
31405,3,140.34,0,1,Urban,1,Phone,1
58088,3,150.76,0,1,Surburban,0,Web,0
44344,2,67.97,1,0,Surburban,1,Phone,1
...,...,...,...,...,...,...,...,...
62570,3,276.21,0,1,Urban,0,Phone,1
38158,4,43.49,0,1,Urban,0,Phone,1
860,8,492.53,1,1,Rural,1,Phone,1
15795,5,72.28,1,0,Urban,1,Web,1


In [232]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44800 entries, 9656 to 56422
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        44800 non-null  int64  
 1   history        44800 non-null  float64
 2   used_discount  44800 non-null  int64  
 3   used_bogo      44800 non-null  int64  
 4   zip_code       44800 non-null  object 
 5   is_referral    44800 non-null  int64  
 6   channel        44800 non-null  object 
 7   treatment      44800 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.1+ MB


In [233]:
x_train["zip_code"].value_counts()

Surburban    20274
Urban        17824
Rural         6702
Name: zip_code, dtype: int64

In [234]:
x_train["channel"].value_counts()

Web             19772
Phone           19635
Multichannel     5393
Name: channel, dtype: int64

In [237]:
def data_processing(data):
    updata = pd.get_dummies(data.copy(), prefix=["zip_code", "channel"])
    # columns = updata.columns
    updata["recency"] = updata["recency"] / updata["recency"].max()
    updata["history"] = updata["history"] / updata["history"].max()
    # updata = pd.DataFrame(data=updata, columns=columns)
    return updata

In [240]:
x_train = data_processing(x_train)

In [241]:
x_test = data_processing(x_test)

In [242]:
x_train

,recency,history,used_discount,used_bogo,is_referral,treatment,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
9656,0.583333,0.135060,1,0,1,0,0,0,1,0,0,1
63037,0.083333,0.117100,1,0,0,1,0,1,0,1,0,0
31405,0.250000,0.043638,0,1,1,1,0,0,1,0,1,0
58088,0.250000,0.046879,0,1,0,0,0,1,0,0,0,1
44344,0.166667,0.021135,1,0,1,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
62570,0.250000,0.085887,0,1,0,1,0,0,1,0,1,0
38158,0.333333,0.013523,0,1,0,1,0,0,1,0,1,0
860,0.666667,0.153151,1,1,1,1,1,0,0,0,1,0
15795,0.416667,0.022475,1,0,1,1,0,0,1,0,0,1


In [243]:
from sklearn.linear_model import LogisticRegression

In [244]:
model = LogisticRegression()

In [246]:
model.fit(x_train, y_train)

LogisticRegression()

In [253]:
x_test_C = x_test.loc[x_test["treatment"] == 0]
x_test_T = x_test.loc[x_test["treatment"] == 1][0:x_test_C.shape[0]]

In [295]:
preict_probabilities_x_test_T = model.predict_proba(x_test_C).T
preict_probabilities_x_test_C = model.predict_proba(x_test_T).T

In [296]:
preict_x_test_T = preict_probabilities_x_test_T[0].T
preict_x_test_C = preict_probabilities_x_test_C[0].T

In [297]:
preict_x_test_T, preict_x_test_C

(array([0.890264  , 0.95901548, 0.91221138, ..., 0.87989635, 0.83503143,
        0.91599341]),
 array([0.90586527, 0.75092633, 0.84074093, ..., 0.76389886, 0.74724312,
        0.83460673]))

In [ ]:
preict_x_test_T - preict_x_test_C

In [298]:
uplift = preict_x_test_T - preict_x_test_C

In [299]:
uplift

array([-0.01560127,  0.20808915,  0.07147045, ...,  0.11599749,
        0.08778831,  0.08138668])